In [30]:
import speech_recognition as sr
import pyttsx3
import pywhatkit
import openai
import os

In [22]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [23]:
# initialize text-to-speech engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

In [24]:
# define function to speak a response
def speak(text):
    engine.say(text)
    engine.runAndWait()

# define function to listen to user's voice command
def get_command():
    # initialize recognizer and microphone
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        # adjust for ambient noise
        r.adjust_for_ambient_noise(source)
        # listen for user's voice command
        audio = r.listen(source)
        try:
            # recognize speech using Google Speech Recognition
            command = r.recognize_google(audio)
            print("You said: " + command)
        except sr.UnknownValueError:
            speak("Sorry, I did not understand.")
            command = ""
        return command.lower()

In [34]:
# class CreateBot:
#     def __init__(self):
#         system_prompt = "You are a helpful AI Assistant that only answers if its 100% certaint"
#         self.system = system_prompt
#         self.messages = [{'role':'system','content':system_prompt}]

#     def chat(self,command):
#         # print('To terminate the conversation, type "END"')
#         question = command
#         n = 1
#         while "exit" not in question or "stop" not in question or "play" not in question:

#             if question == command and n != 1:
#                 command = get_command()
#             question = command
#             # print('\n')
#             self.messages.append({'role':'user','content':question})
#             response = openai.Completion.create(
#                 model='davinci',
#                 prompt='\n'.join([m['content'] for m in self.messages]),
#                 temperature=0.1,
#                 max_tokens=512,
#                 top_p=1,
#                 frequency_penalty=0.6,
#                 presence_penalty=0.2
#             )
#             content = response.choices[0].text.strip()
#             print(content)
#             speak(content)
#             n =2
#             self.messages.append({'role':'assistant','content':content})


In [39]:
# chat_bot = CreateBot()
# chat_bot.chat('what do you know about the Obama')


In [59]:
# define function to execute voice commands
def run_assistant(chat_bot):
    command = get_command()
    # print(command)
    if "play" in command:
        song = command.split("play ")[1]
        speak("Playing " + song + " on YouTube.")
        pywhatkit.playonyt(song)

    elif "exit" in command or "stop" in command:
        speak("Goodbye!")
        return False

    else:
        exit_var,command = chat_bot.chat(command)
        if exit_var == True:
            return False
        else:
            song = command.split("play ")[1]
            speak("Playing " + song + " on YouTube.")
            pywhatkit.playonyt(song)
    return True



In [27]:
get_command()

Listening...
You said: blah


'blah'

In [60]:
# main program loop
AI_ASSISTANT = CreateBot(system_prompt="You are a helpful AI Assistant that only answers if its 100% certaint")
speak("Hello! How can I assist you?")
while True:
    if not run_assistant(AI_ASSISTANT):
        break

Listening...
You said: Obama who is Obama


Barack Obama is a former President of the United States. He served as the 44th President from 2009 to 2017.
Listening...
You said: play BTS
Listening...
You said: nice song exit exit


In [57]:
class CreateBot:

    def __init__(self,system_prompt):

        self.system = system_prompt
        self.messages = [{'role':'system','content':system_prompt}]

    # def chat(self):
    def chat(self,command):

        # print('To terminate the conversation, type "END"')
        # question = ''

        question = command
        n = 1
        exit_var = False
        while "exit" not in question or "stop" not in question or "play" not in question:

            
            
            if question == command and n != 1:
                command = get_command()
            
            question = command
            
            if "exit" in command or "stop" in question:
                exit_var = True
            elif "play" in command:
                break

        # while question != 'END':

            # user input
            # question = input("")
            print('\n')
            # ad user input to messages
            self.messages.append({'role':'user','content':question})
            #grab response
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages = self.messages,
                temperature=0.1,
                # max_tokens=512,
                max_tokens=126,
                top_p=1,
                frequency_penalty=0.6,
                presence_penalty=0.2
            )
            #grab response 
            content = response['choices'][0]['message']['content']
            # print('\n')
            print(content)
            speak(content)
            # print('\n')
            n=2
            

            self.messages.append({'role':'assistant','content':content})

        return exit_var,command

            

In [37]:
AI_ASSISTANT = CreateBot(system_prompt="You are a helpful AI Assistant that only answers if its 100% certaint")

In [38]:
AI_ASSISTANT.chat()

To terminate the conversation, type "END"






Exception ignored in: <function BSTR.__del__ at 0x00000188CF118280>
Traceback (most recent call last):
  File "c:\Users\kazos\anaconda3\lib\site-packages\comtypes\__init__.py", line 999, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt: 
